In [26]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [27]:
import os
import pandas as pd
import numpy as np
import sys
from tqdm import tqdm
sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import dlc_utils

In [28]:
from importlib import reload
reload(dlc_utils)

<module 'dlc_utils' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/dlc_utils.py'>

In [29]:
dlc_directory = '/volumes/My_Passport/dlc_analysis/behavcamvideos/open_field_miniscope/'

In [30]:
experiment_folders = [folder for folder in os.listdir(dlc_directory) if '_H' in folder]

In [31]:
# exclude 'GRIN039_H12_M33_S29'
experiment_folders[22:]

['GRIN038_H13_M37_S23',
 'GRIN038_H15_M39_S40',
 'GRIN039_H12_M26_S54',
 'GRIN039_H12_M33_S29',
 'GRIN039_H14_M8_S53',
 'GRIN041_H12_M54_S49',
 'GRIN041_H16_M35_S12']

In [34]:
# GRIN018_H17_M41_S43
experiment_folders = ['GRIN038_H15_M39_S40',
 'GRIN039_H12_M26_S54',
 'GRIN039_H14_M8_S53',
 'GRIN041_H12_M54_S49',
 'GRIN041_H16_M35_S12']

In [ ]:
for folder in tqdm(experiment_folders):
    output_directory = dlc_directory+folder+'/'
    session_name = folder
    print(folder)
    dlc_output_files = os.listdir(output_directory)
    dlc_output_files.sort()
    
    names = []
    dlc_dfs = []
    for output_file in dlc_output_files:
        if output_file.endswith('000.h5'):
            print(output_file)
            dlc_output = pd.read_hdf(output_directory + output_file)
            dlc_output = dlc_utils.calculate_centroid(dlc_output)
            dlc_output.head()
            coordinates_delta_df = dlc_utils.difference_df(dlc_output)
            df_columns = list(dlc_output.columns)
            print('calculating velocity')
            velocity_df = pd.DataFrame(np.transpose(np.array([np.array([dlc_utils.velocity(coordinates_delta_df[body_part]['x'].values[frame],coordinates_delta_df[body_part]['y'].values[frame]) for frame in range(len(coordinates_delta_df))]) for body_part in list(set([df_columns[item][1] for item in range(len(df_columns))]))])), 
                                       columns=list(set([df_columns[item][1] for item in range(len(df_columns))]))) 
            for body_part in list(set([df_columns[item][1] for item in range(len(df_columns))])):
                velocity_df[body_part+'_likelihood']=dlc_output[df_columns[0][0]][body_part]['likelihood'].values
            
            names.append(output_file)
            dlc_dfs.append(velocity_df)

    session_compiled = pd.concat(dlc_dfs, keys=names)

    #save compiled session 
    session_compiled.to_hdf(output_directory+session_name+'_compiled_session_04142020.h5', key='df')

    #create behavior data aligned to the msCam frames
    aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+session_name+'_compiled_session_04142020.h5', output_directory+'timestamp.dat')
    aligned.reset_index(inplace=True)
    aligned['msCamFrame'] = aligned.index.values
    aligned = aligned.set_index(pd.to_timedelta(np.linspace(0, len(aligned)*(1/20), len(aligned)), unit='s'), drop=False)

    #smooth data by downsampling and interpolating 
    interpolated = dlc_utils.downsample_and_interpolate(aligned, '.05S', '1S', 'linear')
    interpolated.to_csv(output_directory+session_name+'_dlc_tracking_foranalysis_04142020.csv')

    output_directory+session_name+'_dlc_tracking_foranalysis.csv'



  0%|          | 0/5 [00:00<?, ?it/s]

GRIN038_H15_M39_S40
behavCam01DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam02DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam03DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam04DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam05DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam06DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam07DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam08DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam09DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam10DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam11DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam12DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam




  0%|          | 0/58470 [00:00<?, ?it/s]


  0%|          | 72/58470 [00:00<01:21, 717.87it/s]


  0%|          | 143/58470 [00:00<01:21, 712.76it/s]


  0%|          | 214/58470 [00:00<01:22, 709.44it/s]


  0%|          | 284/58470 [00:00<01:22, 705.90it/s]


  1%|          | 355/58470 [00:00<01:22, 705.13it/s]


  1%|          | 426/58470 [00:00<01:22, 705.06it/s]


  1%|          | 495/58470 [00:00<01:22, 699.49it/s]


  1%|          | 565/58470 [00:00<01:22, 699.16it/s]


  1%|          | 635/58470 [00:00<01:22, 698.40it/s]


  1%|          | 705/58470 [00:01<01:22, 696.58it/s]


  1%|▏         | 773/58470 [00:01<01:24, 683.06it/s]


  1%|▏         | 842/58470 [00:01<01:24, 684.69it/s]


  2%|▏         | 911/58470 [00:01<01:24, 683.65it/s]


  2%|▏         | 980/58470 [00:01<01:24, 684.20it/s]


  2%|▏         | 1050/58470 [00:01<01:23, 687.08it/s]


  2%|▏         | 1120/58470 [00:01<01:23, 689.13it/s]


  2%|▏         | 1190/58470 [00:01<01:23, 689.85it/s]


  2%|▏         |

 14%|█▎        | 8035/58470 [00:12<01:27, 575.85it/s]


 14%|█▍        | 8093/58470 [00:12<01:27, 576.90it/s]


 14%|█▍        | 8151/58470 [00:13<01:27, 576.67it/s]


 14%|█▍        | 8209/58470 [00:13<01:27, 573.51it/s]


 14%|█▍        | 8267/58470 [00:13<01:28, 568.99it/s]


 14%|█▍        | 8324/58470 [00:13<01:28, 568.99it/s]


 14%|█▍        | 8381/58470 [00:13<01:28, 568.39it/s]


 14%|█▍        | 8438/58470 [00:13<01:28, 567.07it/s]


 15%|█▍        | 8495/58470 [00:13<01:29, 556.82it/s]


 15%|█▍        | 8552/58470 [00:13<01:29, 558.76it/s]


 15%|█▍        | 8610/58470 [00:13<01:28, 562.56it/s]


 15%|█▍        | 8667/58470 [00:13<01:28, 564.12it/s]


 15%|█▍        | 8724/58470 [00:14<01:28, 564.29it/s]


 15%|█▌        | 8781/58470 [00:14<01:28, 563.29it/s]


 15%|█▌        | 8838/58470 [00:14<01:28, 561.02it/s]


 15%|█▌        | 8895/58470 [00:14<01:32, 534.81it/s]


 15%|█▌        | 8949/58470 [00:14<01:34, 525.74it/s]


 15%|█▌        | 9004/58470 [00:14<01:33, 530.99

 25%|██▍       | 14522/58470 [00:25<01:34, 467.31it/s]


 25%|██▍       | 14569/58470 [00:25<01:34, 464.27it/s]


 25%|██▍       | 14616/58470 [00:25<01:34, 461.64it/s]


 25%|██▌       | 14664/58470 [00:25<01:34, 464.54it/s]


 25%|██▌       | 14713/58470 [00:25<01:33, 470.18it/s]


 25%|██▌       | 14762/58470 [00:26<01:32, 474.65it/s]


 25%|██▌       | 14810/58470 [00:26<01:32, 474.26it/s]


 25%|██▌       | 14860/58470 [00:26<01:30, 479.58it/s]


 25%|██▌       | 14909/58470 [00:26<01:30, 481.87it/s]


 26%|██▌       | 14958/58470 [00:26<01:30, 479.94it/s]


 26%|██▌       | 15007/58470 [00:26<01:30, 482.91it/s]


 26%|██▌       | 15056/58470 [00:26<01:32, 467.73it/s]


 26%|██▌       | 15103/58470 [00:26<01:33, 466.29it/s]


 26%|██▌       | 15150/58470 [00:26<01:42, 423.35it/s]


 26%|██▌       | 15194/58470 [00:26<01:44, 415.82it/s]


 26%|██▌       | 15237/58470 [00:27<01:43, 419.65it/s]


 26%|██▌       | 15280/58470 [00:27<01:45, 408.28it/s]


 26%|██▌       | 15325/58470 [0

...

 74%|███████▍  | 44647/60418 [05:29<15:43, 16.71it/s]


 74%|███████▍  | 44669/60418 [05:29<11:21, 23.12it/s]


 74%|███████▍  | 44688/60418 [05:31<13:17, 19.72it/s]


 74%|███████▍  | 44715/60418 [05:31<09:44, 26.88it/s]


 74%|███████▍  | 44730/60418 [05:31<09:00, 29.02it/s]


 74%|███████▍  | 44774/60418 [05:31<06:28, 40.32it/s]


 74%|███████▍  | 44800/60418 [05:31<04:49, 53.98it/s]


 74%|███████▍  | 44823/60418 [05:32<05:45, 45.08it/s]


 74%|███████▍  | 44840/60418 [05:32<04:30, 57.54it/s]


 74%|███████▍  | 44857/60418 [05:33<06:22, 40.72it/s]


 74%|███████▍  | 44870/60418 [05:35<16:03, 16.13it/s]


 74%|███████▍  | 44904/60418 [05:36<13:28, 19.18it/s]


 74%|███████▍  | 44953/60418 [05:36<09:34, 26.94it/s]


 74%|███████▍  | 44973/60418 [05:37<09:51, 26.12it/s]


 74%|███████▍  | 44988/60418 [05:37<09:16, 27.73it/s]


 75%|███████▍  | 45030/60418 [05:37<06:41, 38.30it/s]


 75%|███████▍  | 45055/60418 [05:37<05:00, 51.13it/s]


 75%|███████▍  | 45075/60418 [05:42<21:34, 11.85

 80%|████████  | 48400/60418 [06:56<03:29, 57.24it/s]


 80%|████████  | 48417/60418 [06:57<03:51, 51.82it/s]


 80%|████████  | 48430/60418 [06:59<08:13, 24.30it/s]


 80%|████████  | 48440/60418 [07:01<25:17,  7.89it/s]


 80%|████████  | 48447/60418 [07:02<21:08,  9.44it/s]


 80%|████████  | 48453/60418 [07:02<18:32, 10.75it/s]


 80%|████████  | 48458/60418 [07:02<14:14, 13.99it/s]


 80%|████████  | 48471/60418 [07:02<10:25, 19.10it/s]


 80%|████████  | 48501/60418 [07:02<07:28, 26.54it/s]


 80%|████████  | 48524/60418 [07:02<05:30, 35.98it/s]


 80%|████████  | 48541/60418 [07:02<04:12, 47.10it/s]


 80%|████████  | 48581/60418 [07:03<03:04, 64.03it/s]


 80%|████████  | 48621/60418 [07:03<02:22, 83.00it/s]


 81%|████████  | 48646/60418 [07:03<02:26, 80.13it/s]


 81%|████████  | 48666/60418 [07:07<12:48, 15.28it/s]


 81%|████████  | 48680/60418 [07:08<11:52, 16.48it/s]


 81%|████████  | 48691/60418 [07:08<10:26, 18.72it/s]


 81%|████████  | 48699/60418 [07:08<08:39, 22.56

 87%|████████▋ | 52372/60418 [07:34<00:18, 428.31it/s]


 87%|████████▋ | 52424/60418 [07:34<00:17, 451.02it/s]


 87%|████████▋ | 52472/60418 [07:34<00:24, 325.85it/s]


 87%|████████▋ | 52524/60418 [07:34<00:21, 366.65it/s]


 87%|████████▋ | 52567/60418 [07:34<00:32, 239.22it/s]


 87%|████████▋ | 52611/60418 [07:34<00:28, 276.77it/s]


 87%|████████▋ | 52658/60418 [07:35<00:24, 314.38it/s]


 87%|████████▋ | 52709/60418 [07:35<00:21, 354.17it/s]


 87%|████████▋ | 52752/60418 [07:35<00:35, 213.81it/s]


 87%|████████▋ | 52786/60418 [07:35<00:51, 149.28it/s]


 87%|████████▋ | 52812/60418 [07:36<00:53, 141.53it/s]


 87%|████████▋ | 52841/60418 [07:36<00:45, 167.12it/s]


 88%|████████▊ | 52875/60418 [07:36<00:38, 197.06it/s]


 88%|████████▊ | 52904/60418 [07:36<00:44, 170.71it/s]


 88%|████████▊ | 52927/60418 [07:36<01:11, 104.41it/s]


 88%|████████▊ | 52945/60418 [07:37<01:43, 72.09it/s] 


 88%|████████▊ | 52974/60418 [07:37<01:20, 92.95it/s]


 88%|████████▊ | 53023/60418 [07

 96%|█████████▌| 57771/60418 [07:54<00:11, 235.70it/s]


 96%|█████████▌| 57823/60418 [07:54<00:09, 281.75it/s]


 96%|█████████▌| 57861/60418 [07:55<00:31, 80.19it/s] 


 96%|█████████▌| 57889/60418 [07:55<00:25, 98.90it/s]


 96%|█████████▌| 57945/60418 [07:55<00:19, 124.85it/s]


 96%|█████████▌| 58002/60418 [07:55<00:14, 162.90it/s]


 96%|█████████▌| 58039/60418 [07:56<00:20, 113.42it/s]


 96%|█████████▌| 58071/60418 [07:56<00:16, 140.21it/s]


 96%|█████████▌| 58125/60418 [07:56<00:12, 179.82it/s]


 96%|█████████▋| 58164/60418 [07:56<00:10, 214.07it/s]


 96%|█████████▋| 58201/60418 [07:56<00:09, 239.08it/s]


 96%|█████████▋| 58242/60418 [07:56<00:08, 271.80it/s]


 96%|█████████▋| 58297/60418 [07:57<00:06, 320.10it/s]


 97%|█████████▋| 58340/60418 [07:57<00:06, 317.78it/s]


 97%|█████████▋| 58379/60418 [07:57<00:06, 304.02it/s]


 97%|█████████▋| 58415/60418 [07:57<00:08, 245.87it/s]


 97%|█████████▋| 58458/60418 [07:57<00:06, 281.67it/s]


 97%|█████████▋| 58512/60418 [07

GRIN041_H16_M35_S12
behavCam01DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam02DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam03DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam04DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam05DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam06DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam07DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam08DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam09DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam10DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam11DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam12DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5
calculating velocity
behavCam

In [ ]:
#fixing errors
GRIN012_H17_M46_S35_error
GRIN033_H13_M42_S33
GRIN033_H14_M34_S32_01162020

In [17]:
#aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+'GRIN012_H17_M46_S35_compiled_session_04142020.h5', output_directory+'timestamp.dat')
dlc_analysis = pd.read_hdf(output_directory+'GRIN012_H17_M46_S35_compiled_session_04142020.h5')

In [21]:
frame_clock_df = pd.read_table(output_directory+'timestamp.dat')

In [20]:
dlc_full = dlc_full.reset_index()

In [23]:
msCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 0].set_index('frameNum')
behavCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 1].set_index('frameNum')
msCam_timestamps['sysClock'][1] = 0
behavCam_timestamps['sysClock'][1] = 0

In [27]:
msCam_timestamps

,camNum,sysClock,buffer
frameNum,,,
1,0,0,1
2,0,48,1
3,0,102,1
4,0,152,1
5,0,203,1
...,...,...,...
18770,0,939217,1
18771,0,939267,1
18772,0,939316,1


In [26]:
behavCam_timestamps

,camNum,sysClock,buffer
frameNum,,,
1,1,0,2
2,1,26,1
3,1,60,1
4,1,98,1
5,1,124,1
...,...,...,...
28164,1,939291,1
28165,1,939323,1
28166,1,939355,1


In [25]:
msCam_timestamps = dlc_utils.align_behavior_data(msCam_timestamps, behavCam_timestamps)
msCam_timestamps.reset_index(inplace=True)





  0%|          | 0/18388 [00:00<?, ?it/s]



  0%|          | 66/18388 [00:00<00:27, 659.72it/s]



  1%|          | 136/18388 [00:00<00:27, 671.23it/s]



  1%|          | 204/18388 [00:00<00:27, 672.73it/s]



  1%|▏         | 272/18388 [00:00<00:26, 673.16it/s]



  2%|▏         | 338/18388 [00:00<00:26, 668.90it/s]



  2%|▏         | 403/18388 [00:00<00:27, 660.82it/s]



  3%|▎         | 466/18388 [00:00<00:27, 649.91it/s]



  3%|▎         | 527/18388 [00:00<00:28, 636.60it/s]



  3%|▎         | 587/18388 [00:00<00:29, 599.15it/s]



  4%|▎         | 651/18388 [00:01<00:29, 609.75it/s]



  4%|▍         | 711/18388 [00:01<00:29, 604.56it/s]



  4%|▍         | 771/18388 [00:01<00:30, 582.50it/s]



  5%|▍         | 833/18388 [00:01<00:29, 592.17it/s]



  5%|▍         | 899/18388 [00:01<00:28, 608.87it/s]



  5%|▌         | 963/18388 [00:01<00:28, 617.58it/s]



  6%|▌         | 1026/18388 [00:01<00:28, 618.44it/s]



  6%|▌         | 1088/18388 [00:01<00:28, 606.31it/s]



 65%|██████▌   | 11978/18388 [00:24<00:14, 447.40it/s]



 65%|██████▌   | 12023/18388 [00:24<00:14, 445.16it/s]



 66%|██████▌   | 12073/18388 [00:24<00:13, 458.06it/s]



 66%|██████▌   | 12123/18388 [00:24<00:13, 468.52it/s]



 66%|██████▌   | 12172/18388 [00:24<00:13, 474.70it/s]



 66%|██████▋   | 12221/18388 [00:24<00:12, 477.12it/s]



 67%|██████▋   | 12270/18388 [00:24<00:12, 480.41it/s]



 67%|██████▋   | 12321/18388 [00:25<00:12, 486.36it/s]



 67%|██████▋   | 12371/18388 [00:25<00:12, 489.00it/s]



 68%|██████▊   | 12422/18388 [00:25<00:12, 494.39it/s]



 68%|██████▊   | 12472/18388 [00:25<00:11, 493.29it/s]



 68%|██████▊   | 12522/18388 [00:25<00:11, 493.13it/s]



 68%|██████▊   | 12573/18388 [00:25<00:11, 496.25it/s]



 69%|██████▊   | 12624/18388 [00:25<00:11, 497.94it/s]



 69%|██████▉   | 12675/18388 [00:25<00:11, 500.62it/s]



 69%|██████▉   | 12727/18388 [00:25<00:11, 505.58it/s]



 69%|██████▉   | 12779/18388 [00:25<00:11, 509.12it/s]



 70%|██████▉  

KeyError: 18369





100%|█████████▉| 18312/18388 [00:48<00:00, 644.03it/s]